In [106]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/annabelitong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/annabelitong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/annabelitong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/annabelitong/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [107]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.casual import TweetTokenizer
t = TweetTokenizer()
def clean_news_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # convert to lowercase
    text = text.lower()
    
    # tokenize the text
    tokens = TweetTokenizer().tokenize(text)
    
    # remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # perform lemmatization
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # join the tokens back into a string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


In [108]:
def get_nba_text(url):

    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser") 
    content = soup.find_all("p")
    text = ""
    for i in content:
        text += i.text
    clean_text = text.replace('\n', ' ').replace('\xa0', '')
    return clean_text


In [109]:
from nltk.tokenize.casual import TweetTokenizer
t = TweetTokenizer()

def preprocess_text(text):
    # convert the text to lowercase
    text = text.lower()
    # remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    # tokenize the text
    tokens = TweetTokenizer().tokenize(text)
    # remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return filtered_tokens

In [110]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def get_sentiment_score_nltk(content):
    # Preprocess the text data
    # Tokenize the preprocessed text
    # Combine the title and content text

    # Apply sentiment analysis
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(content)
    
    # Return the sentiment score
    return scores['compound']


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/annabelitong/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [111]:
from textblob import TextBlob

def get_sentiment_score_tblob(text):
    # Create a TextBlob object from the text
    blob = TextBlob(text)
    
    # Get the sentiment score using TextBlob's sentiment analysis
    sentiment_score = blob.sentiment.polarity
    
    # Return the sentiment score
    return sentiment_score


In [112]:
def get_nba_processed_clean_text(url):

    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser") 
    content = soup.find_all("p")
    text = ""
    for i in content:
        text += i.text
    clean_text = text.replace('\n', ' ').replace('\xa0', '')
    processed_clean_text = preprocess_text(clean_text)
    return processed_clean_text


In [113]:
from collections import Counter

def count_words(cleaned_list):
    word_counts = Counter(cleaned_list)
    top_5 = word_counts.most_common(5)

    return top_5


In [114]:
def add_relevance_df(nba_news_df):
    url_list = list(nba_news_df["url"])
    nltk_primary = []
    tblob_primary = []
    nltk_processed = []
    tblob_processed = []
    top_words = []
    
    for i in url_list:
        text_1 = get_nba_text(i)
        nltk_score = get_sentiment_score_nltk(text_1)
        tblob_score = get_sentiment_score_tblob(text_1)
        nltk_primary.append(nltk_score)
        tblob_primary.append(tblob_score)

        list_text = preprocess_text(text_1)
        top_five = count_words(list_text)
        top_words.append(top_five)
        full_text = ' '.join(list_text)
        nltk_sec = get_sentiment_score_nltk(full_text)
        tblob_sec = get_sentiment_score_tblob(full_text)
        nltk_processed.append(nltk_sec)
        tblob_processed.append(tblob_sec)

    nba_news_df["nltk_primary"] = nltk_primary
    nba_news_df["tblob_primary"] = tblob_primary
    nba_news_df["nltk_processed"] = nltk_processed
    nba_news_df["tblob_processed"] = tblob_processed
    nba_news_df["top_words"] = top_words

    return nba_news_df
        






        
        


In [115]:
nba_600_author = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_author/news_600_author.csv')
nba_1200_author = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_author/news_1200_author.csv')
#nba_1600 = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_author/news_1600_author.csv')


In [116]:
nba_600_author_1 = nba_600_author.copy()

In [117]:
df_1 = nba_600_author_1[:10]

In [118]:
relevance = add_relevance_df(df_1)

/var/folders/9g/wg6kygl97c58z_h6ljqdr8g00000gn/T/ipykernel_95938/2434662114.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_news_df["nltk_primary"] = nltk_primary
/var/folders/9g/wg6kygl97c58z_h6ljqdr8g00000gn/T/ipykernel_95938/2434662114.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_news_df["tblob_primary"] = tblob_primary
/var/folders/9g/wg6kygl97c58z_h6ljqdr8g00000gn/T/ipykernel_95938/2434662114.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [122]:
relevance

,Unnamed: 0,title,time,url,author,nltk_primary,tblob_primary,nltk_processed,tblob_processed,top_words
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...","October 24, 2020",http://global.nba.com/news/bucks-george-hill-r...,Steve Megargee,0.9599,0.079299,0.9713,0.088752,"[(’, 14), (hill, 10), (“, 9), (”, 8), (milwauk..."
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...","October 24, 2020",http://global.nba.com/news/nba-wnba-ramp-up-ef...,Brian Martin,0.9997,0.212208,0.9994,0.177785,"[(nba, 20), (vote, 16), (voting, 13), (electio..."
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...","October 23, 2020",http://global.nba.com/news/pat-riley-discusses...,Tim Reynolds | Associated Press,0.9851,0.115362,0.9878,0.132083,"[(’, 9), (riley, 8), (season, 8), (“, 7), (”, 7)]"
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,"October 23, 2020",http://global.nba.com/news/report-amare-stoude...,NBA News,0.9001,0.109621,0.9001,0.089075,"[(nba, 7), (stoudemire, 5), (’, 4), (league, 4..."
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,"October 22, 2020",http://global.nba.com/news/espn-to-host-virtua...,Official Release,0.4939,0.204545,0.4939,0.204545,"[(espn, 7), (nba, 3), (october, 2), (2020, 2),..."
5,5,LeBron James Ranks No. 1 In NBA 2K21’s Player ...,"October 22, 2020",http://global.nba.com/news/lebron-james-ranks-...,NBA News,0.8555,-0.002041,0.8555,0.010714,"[(•, 15), (nba, 6), (2020, 3), (james, 3), (2k..."
6,6,Home Of Denver Nuggets Renamed Ball Arena,"October 22, 2020",http://global.nba.com/news/home-of-denver-nugg...,The Assocaited Press,0.9843,0.165000,0.9843,0.161538,"[(’, 5), (ball, 5), (arena, 4), (home, 3), (co..."
7,7,Consensus Mock Draft: LaMelo Ball Leapfrogs An...,"October 22, 2020",http://global.nba.com/news/consensus-mock-draf...,"Drew Packham, NBA.com",-0.3612,0.131528,0.9925,0.083957,"[(ball, 18), (3, 18), (10, 17), (5, 16), (1, 15)]"
8,8,New Clippers Coach Tyronn Lue: ‘It Feels Good ...,"October 21, 2020",http://global.nba.com/news/new-clippers-coach-...,Beth Harris,0.9986,0.206805,0.9973,0.225282,"[(’, 28), (“, 19), (”, 19), (lue, 17), (said, ..."
9,9,Daniel Theis Medical Update,"October 21, 2020",http://global.nba.com/news/daniel-theis-medica...,Official Release,0.5859,0.126573,0.5859,0.101573,"[(october, 2), (21, 2), (2020, 2), (celtics, 2..."


In [123]:
url = nba_600_author_1[:1]["url"]
url[0]


'http://global.nba.com/news/bucks-george-hill-returns-to-milwaukee-encourages-voting/'

In [124]:
text = get_nba_processed_clean_text(url[0])
text

['october',
 '24',
 '2020',
 'steve',
 'megargee',
 'milwaukee',
 'ap',
 '—',
 'milwaukee',
 'bucks',
 'guard',
 'george',
 'hill',
 'said',
 'importance',
 'wisconsin',
 'upcoming',
 'election',
 'caused',
 'fly',
 'back',
 'weekend',
 'assist',
 'local',
 'voting',
 'drivehill',
 'lives',
 'san',
 'antonio',
 'worked',
 'alongside',
 '50',
 'volunteers',
 'milwaukeebased',
 'organization',
 'common',
 'ground',
 'walking',
 'local',
 'neighborhoods',
 'saturday',
 'remind',
 'people',
 'vote',
 '“',
 'state',
 'could',
 'make',
 'break',
 'election',
 '”',
 'hill',
 'said',
 '“',
 'spend',
 'couple',
 'dollars',
 'get',
 'flight',
 'come',
 'back',
 'get',
 'knock',
 'doors',
 'go',
 'around',
 'city',
 'show',
 'care',
 'much',
 'care',
 'show',
 'need',
 'need',
 'need',
 'voices',
 'need',
 'communities',
 'better',
 'things',
 'like',
 '’',
 'worth',
 'end',
 'day',
 '”',
 'hill',
 'wore',
 'shirt',
 'message',
 '“',
 'vote',
 '”',
 'bucks',
 'logo',
 'plus',
 'mask',
 'message',